# motivation

## Preprogress

In [ ]:
import pandas as pd
import glob
import numpy as np
import re
import os


folder_path = 'Path/to/inference'
folder_name = "inference_folder"
model_name = folder_name.split('_')[-1]
kind_exp = folder_name.split('_')[-2]

file_paths = glob.glob(folder_path + folder_name + "/result_index_*.csv")

def extract_index(path):
    match = re.search(r"result_index_(\d+)_", os.path.basename(path))
    return int(match.group(1)) if match else 1e9   #
file_paths = sorted(file_paths, key=extract_index)


merged_df = pd.concat([pd.read_csv(fp) for fp in file_paths], ignore_index=True)

profile_columns = merged_df.columns[:5]
res_columns = merged_df.columns[list(range(5,35,2))]
rea_columns = merged_df.columns[list(range(6,36,2))]

print(merged_df.shape) 
merged_df.head(2)


In [ ]:
def process_text_result(text):
    if text is None:
        return ""
    text = str(text)
    filt_str = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9=()（）]', '', text)
    
    match = re.search(r'选择(.*?)$', filt_str) 
    if match:
        choice = match.group(1)

    else:
        print("choice error")
        choice = ''
    
    return choice


def process_text_series(text_serires,both_flag):
    choice_series, thinking_answer_serires = [],[]
    for text_item in enumerate(text_serires):
        if both_flag == 1: 
            # _,choice,thinking_answer = process_text(text_item[1])
            pass
        elif both_flag==0:
            choice = process_text_result(text_item[1])
            thinking_answer = None
        elif both_flag == 2: 
            # _,choice,thinking_answer = process_text_trained(text_item[1])
            pass
        choice_series.append(choice)
        thinking_answer_serires.append(thinking_answer)

    return choice_series,thinking_answer_serires

In [ ]:

choice_columns,think_columns = [],[]
for question_id, question_column in enumerate(res_columns):
    new_col_name = str(question_id)+'_chocice'
    
    choice_columns.append(new_col_name)        
    choice, thinking_answer = process_text_series(merged_df[question_column],0)
    merged_df[new_col_name] = choice

    if kind_exp != 'direct': 
        new_col_name_think = str(question_id) + '_thinking'
        think_columns.append(new_col_name_think)
        merged_df[new_col_name_think] = merged_df.iloc[:,int(question_column)+1]

print(merged_df.shape)

merged_df.to_csv(folder_path + folder_name + "/merged_result.csv", index=False)

merged_df.head()

In [ ]:
import numpy as np
import json

question_path = './Code/Data/SBR-Question-list.json'
with open(question_path, 'r', encoding='utf-8') as file:
    question_dict = json.load(file)
question_list = []
for topic,topic_value in question_dict.items():
    for questin_id,question_value in topic_value.items():
        question_list.append(question_value['question'])

for question_id,question in enumerate(question_list):
    option_len = len(question['option'])
    option_to_num = {option.replace('，', ''): idx for idx, option in enumerate(question['option'])}
    if question_id in [6,7,8]:
        # 道德困境单独加
        option_to_num['yes']=0
        option_to_num['no']=1
    if question_id == 9:
        new_dict = option_to_num.copy()
        for key, value in option_to_num.items():
            number = key.split('=')[1]  
            new_dict[number] = value
        option_to_num = new_dict.copy()
    

    def custom_map(value):
        if not isinstance(value, str):  
            return value  
        for key, num in option_to_num.items():
            if key in value:
                return num
        return np.nan  
    
    
    merged_df[choice_columns[question_id]] = merged_df[choice_columns[question_id]].apply(custom_map)
    
    merged_df[choice_columns[question_id]] = merged_df[choice_columns[question_id]] / (option_len - 1)
    


In [ ]:

if kind_exp != 'direct':
    missing_rows = merged_df.loc[:,choice_columns][merged_df.loc[:,choice_columns].isnull().any(axis=1)]
    missing_count_per_row = missing_rows.isnull().sum(axis=1)
    print(missing_count_per_row)
    total_missing_count = missing_count_per_row.sum()
    print(total_missing_count)
else :
    missing_rows = merged_df[merged_df.isnull().any(axis=1)]
    missing_count_per_row = missing_rows.isnull().sum(axis=1)-15
    print(missing_count_per_row[missing_count_per_row>0])
    total_missing_count = missing_count_per_row.sum()
    print(total_missing_count)

missing_positions = merged_df.isnull()
merged_df.to_csv(folder_path + folder_name + "/merged_choicenum_result.csv", index=False)


## cal matrix

In [ ]:
import json
import numpy as np

question_path = './Code/Data/SBR-Question-list.json'
with open(question_path, 'r', encoding='utf-8') as file:
    question_dict = json.load(file)
    
question_list = []
for topic,topic_value in question_dict.items():
    for questin_id,question_value in topic_value.items():
        question_list.append(question_value['question'])

option_num_list = [len(list(item['option'])) for item in question_list]
max_entropy_list = [np.log2(n) for n in option_num_list]

all_possible_values_list = []

for question_id,question in enumerate(question_list):
    option_len = len(question['option'])
    all_possible_values_list.append([i/(option_len-1) for i in range(option_len)])

In [ ]:
import pandas as pd
import numpy as np
def load_data(result_path):
    profile_columns = ['0','1','2','3','4']
    df = pd.read_csv(result_path+'/merged_choicenum_result.csv')
    for col in profile_columns:
        df[col] = df[col].fillna(-1).astype(int) 
    return df

In [ ]:
import numpy as np 

def calculate_entropy(series):
    series = series.dropna()
    counts = series.value_counts()
    probabilities = counts / counts.sum()
    entropy = -np.sum(probabilities * np.log2(probabilities))
    if entropy==-0.00:
        entropy = 0
    return entropy

def calculate_rmse(series,human_series): 
    if len(series) != len(human_series):
        raise ValueError("different length")
   
    mask = ~np.isnan(series) & ~np.isnan(human_series)  
    if not np.any(mask): 
        print("no data")
        return np.nan
    series = series[mask]
    human_series = human_series[mask]

    squared_diff = (series - human_series) ** 2
    rmse = np.sqrt(np.mean(squared_diff))
    return rmse


def Cal_enr_kl_group(df,non_human,human_df):
  
    choice_columns = [str(i)+'_chocice' for i in range(0,15)]
    result_dict = {'entropies':{},'rmse':{}}


    for i,choice_column in enumerate(choice_columns):
        temp_en = calculate_entropy(df[choice_column])/max_entropy_list[i]
        result_dict['entropies'][i] = temp_en
    average_en = np.nanmean(list(result_dict['entropies'].values()))
    result_dict['entropies']['average'] = average_en

    for i,choice_column in enumerate(choice_columns):
        temp_dif = calculate_rmse(df[choice_column],human_df[choice_column])
        result_dict['rmse'][i] = temp_dif
    average_rmse = np.nanmean(list(result_dict['rmse'].values()))
    result_dict['rmse']['average'] = average_rmse
    
    return result_dict


In [ ]:
def save_en_kl(model_name,result_dict,save_path,non_human):
    with open(save_path+'/motivation_result.json', "w", encoding="utf-8") as f:
        json.dump(result_dict, f, ensure_ascii=False, indent=2)
    print(f'model：{model_name} \n en:{result_dict["entropies"]["average"]:.3f}\n recall:{result_dict["recall"]:.3f}\n acc:{1-result_dict["rmse"]["average"]:.3f}')
    return result_dict


In [ ]:

profile_columns = ['0','1','2','3','4']
choice_columns = [str(i)+'_chocice' for i in range(0,15)]
human_df = pd.read_csv("Path/To/Humandata")
human_df['all'] = np.ones(human_df.shape[0])

BASE_FOLDER_PATH= 'Path/to/inference'
BASE_MODEL_NAME= ['inference_folder',] 
for base_model in BASE_MODEL_NAME:
    base_result_path = BASE_FOLDER_PATH+base_model
    base_df = load_data(base_result_path)
    direct_result = Cal_enr_kl_group(base_df,True,human_df) 
    result_csv = save_en_kl(base_model,direct_result,base_result_path,True)